# Import packages

In [1]:
import os
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
rcParams['font.family'] = 'sans-serif'
directory   = '/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS'

# Read mean KmKe and PeKe

In [2]:
dataset1   = sio.loadmat(directory+'/Figures/Figure2_mean.mat')
bran_lon   = dataset1['bran_lon'][:,:]
bran_lat   = dataset1['bran_lat'][:,:]
KmKe_mean  = dataset1['bran_KmKe_mean'][:,:]*1e4
PeKe_mean  = dataset1['bran_PeKe_mean'][:,:]*1e4

#################################################################################################
dataset1 = xr.open_dataset(directory+'/BRAN/trends/KmKe/BRAN_KmKe_trends_KC.nc')
dataset2 = xr.open_dataset(directory+'/BRAN/trends/KmKe/BRAN_KmKe_trends_GS.nc')
dataset3 = xr.open_dataset(directory+'/BRAN/trends/PeKe/BRAN_PeKe_trends_KC.nc')
dataset4 = xr.open_dataset(directory+'/BRAN/trends/PeKe/BRAN_PeKe_trends_GS.nc')
dataset5 = xr.open_dataset(directory+'/BRAN/trends/EKE/BRAN_EKE_trends_KC.nc')
dataset6 = xr.open_dataset(directory+'/BRAN/trends/EKE/BRAN_EKE_trends_GS.nc')

KC_KmKe_trends = dataset1.trend.transpose().values*3650*1e4
KC_KmKe_p = dataset1.p.transpose().values
KC_KmKe_p[np.where(KC_KmKe_p > 0.05)] = np.nan

GS_KmKe_trends = dataset2.trend.transpose().values*3650*1e4
GS_KmKe_p = dataset2.p.transpose().values
GS_KmKe_p[np.where(GS_KmKe_p > 0.05)] = np.nan

KC_PeKe_trends = dataset3.trend.transpose().values*3650*1e4
KC_PeKe_p = dataset3.p.transpose().values
KC_PeKe_p[np.where(KC_PeKe_p > 0.05)] = np.nan

GS_PeKe_trends = dataset4.trend.transpose().values*3650*1e4
GS_PeKe_p = dataset4.p.transpose().values
GS_PeKe_p[np.where(GS_PeKe_p > 0.05)] = np.nan

KC_EKE_trends = dataset5.trend.transpose().values*3650*100
KC_EKE_p = dataset5.p.transpose().values
KC_EKE_p[np.where(KC_EKE_p > 0.05)] = np.nan

GS_EKE_trends = dataset6.trend.transpose().values*3650*100
GS_EKE_p = dataset6.p.transpose().values
GS_EKE_p[np.where(GS_EKE_p > 0.05)] = np.nan
#################################################################################################
dataset7 = xr.open_dataset(directory+'/BRAN/BRAN2020_KC_GS_clim_SSH.nc')
bran_ssh = dataset7.clim_ssh.transpose()
KC_ssh   = bran_ssh[200:502,299:451]
GS_ssh   = bran_ssh[1600:1902,299:451]

# Plot the spatial distribution of BRAN EKE trend, mean KmKe and PeKe

In [ ]:
##############################################################################################################
labels      = ['a','b','c','d','e','f']
levels1     = np.linspace(-3,3,50)
levels2     = np.linspace(-2.0,2.0,50)
tick_marks1 = np.linspace(-3,3,7)
tick_marks2 = np.linspace(-2.0,2.0,5) 
cmaps1      = cmocean.cm.balance
fig_ratio   = 0.9
fig         = plt.figure(figsize=(24, 32))
gs          = gridspec.GridSpec(3,2)
labelfont     = 30
padspacescale = 10
labelpadscale = 2
linefont      = 2
scale1        = 1.0
scale2        = 1.0
m_scale       = 50   
##############################################################################################################
for i in range(6):
    if i==0:
        lon       = bran_lon[200:502,299:451]      
        lat       = bran_lat[200:502,299:451]
        var_mean  = KC_EKE_trends
        var_p     = KC_EKE_p
        var_ssh   = KC_ssh
        ssh_level = [0.7]
    elif i==1:
        lon       = bran_lon[1600:1902,299:451]      
        lat       = bran_lat[1600:1902,299:451]   
        var_mean  = GS_EKE_trends
        var_p     = GS_EKE_p
        var_ssh   = GS_ssh
        ssh_level = [0.0]
    elif i==2:
        lon       = bran_lon[200:502,299:451]      
        lat       = bran_lat[200:502,299:451]
        var_mean  = KmKe_mean[200:502,299:451]
    elif i==3:
        lon       = bran_lon[1600:1902,299:451]      
        lat       = bran_lat[1600:1902,299:451]   
        var_mean  = KmKe_mean[1600:1902,299:451]
    elif i==4:
        lon       = bran_lon[200:502,299:451]      
        lat       = bran_lat[200:502,299:451]  
        var_mean  = PeKe_mean[200:502,299:451]        
    else:
        lon       = bran_lon[1600:1902,299:451]      
        lat       = bran_lat[1600:1902,299:451]   
        var_mean  = PeKe_mean[1600:1902,299:451]
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    m   = Basemap(projection='merc',suppress_ticks=True,llcrnrlat=np.nanmin(lat),urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='c')        
    m.drawparallels(np.arange(15,50, 5),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')    
    m.drawmeridians(np.arange(0,360, 5),labels=[0,0,0,1],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')
    m.drawcoastlines(color='0.1',  linewidth=linefont)
    m.fillcontinents(color='0.9', lake_color='white')
    x, y = m(lon, lat)
    yy = np.arange(1, y.shape[0], 4)
    xx = np.arange(1, x.shape[1], 4)
    if i<2:
        points      = np.meshgrid(yy, xx)
        point_index = tuple(points)
        point_x     = x[point_index]
        point_y     = y[point_index]
        point_z     = var_p[point_index]
        point_x     = np.ravel(point_x)
        point_y     = np.ravel(point_y)
        point_z     = np.ravel(point_z)
        point_x     = point_x[~np.isnan(point_z)]
        point_y     = point_y[~np.isnan(point_z)]
        
    if i==0:
        cx,cy = m(135, 46.0)
        CB1 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
        CS1 = m.scatter(point_x,point_y,s=m_scale,c='dimgray',marker='.') 
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=10*padspacescale,family='sans-serif')
    elif i==1:
        cx,cy  = m(275, 46.0)
        CB1 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
        CS1 = m.scatter(point_x,point_y,s=m_scale,c='dimgray',marker='.') 
    elif i==2: 
        b=b+0.04
        cx,cy = m(135, 46.0)
        CB2 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels2,origin='lower',extend='both')
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=10*padspacescale,family='sans-serif')
    elif i==3:
        b=b+0.04
        cx,cy  = m(276.5, 46.0)
        CB2 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels2,origin='lower',extend='both')
    elif i==4:
        b=b+0.08
        cx,cy = m(136.5, 46.0)
        CB3 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=10*padspacescale,family='sans-serif')
        plt.xlabel('Longitude',fontsize=labelfont,labelpad=5*padspacescale,family='sans-serif')
    else:
        b=b+0.08
        cx,cy  = m(276.5, 46.0)
        CB3 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels2,origin='lower',extend='both')
        plt.xlabel('Longitude',fontsize=labelfont,labelpad=5*padspacescale,family='sans-serif')
   
    if np.mod(i,2)==0:
        CS = m.contour(x, y, KC_ssh,[0.7],linewidths=5*linefont,linestyles='solid',colors='tab:gray')
    elif np.mod(i,2)==1:
        CS = m.contour(x, y, GS_ssh,[0.0],linewidths=5*linefont,linestyles='solid',colors='tab:blue')
    ax.set_position([l, b, scale1*w, scale1*h])   
    m.plot(x[0,:],  y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0],  y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    ax.spines['bottom'].set_linewidth(labelpadscale)
    plt.tick_params(axis='x',which='major',bottom='on',left='on',top='on',right='on',\
                length=25,width=2,colors='black',direction='out')
    plt.text(cx, cy,labels[i],color='xkcd:black', fontsize=1.0*labelfont, weight='bold',family='sans-serif')
    m.fillcontinents(color='0.9', lake_color='white')        
cbaxes1     = fig.add_axes([0.92, 0.685, 0.01, 0.16])
cb1 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
cb1.set_ticks(tick_marks1)
cb1.ax.tick_params(labelsize=0.8*labelfont)
cb1.set_label(r'EKE trend' '\n' '(10$^{-2}$ m$^{2}$ s$^{-2}$ per decade)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
cbaxes2     = fig.add_axes([0.92, 0.455, 0.01, 0.16])
cb2 = plt.colorbar(CB2,orientation='vertical',cax = cbaxes2)
cb2.set_ticks(tick_marks2)
cb2.ax.tick_params(labelsize=0.8*labelfont)
cb2.set_label(r'$\overline{KmKe}$ (10$^{-4}$ W m$^{-3}$)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
cbaxes3     = fig.add_axes([0.92, 0.225, 0.01, 0.16])
cb3 = plt.colorbar(CB3,orientation='vertical',cax = cbaxes3)
cb3.set_ticks(tick_marks2)
cb3.ax.tick_params(labelsize=0.8*labelfont)
cb3.set_label(r'$\overline{PeKe}$ (10$^{-4}$ W m$^{-3}$)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
fig.savefig(directory+'/Figures/FigureS3_EKE_trends_kmke_peke.png',dpi=300,bbox_inches = 'tight')